In [1]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
from glob import glob
import IPython.display as ipd
from tqdm import tqdm
import subprocess
plt.style.use('ggplot')
import os
import shutil
import yaml
import json

In [ ]:
"""def copy_files(source, destination):
    files = os.listdir(source)
    with tqdm(total=len(files), desc=f'Copying files from {source} to {destination}') as pbar:
        for filename in files:
            src_file = os.path.join(source, filename)
            dest_file = os.path.join(destination, filename)
            shutil.copy(src_file, dest_file)
            pbar.update(1)"""

In [ ]:
"""os.makedirs('bdd100k',exist_ok=True)
os.makedirs('bdd100k/train',exist_ok=True)
os.makedirs('bdd100k/val',exist_ok=True)
os.makedirs('bdd100k/test',exist_ok=True)
os.makedirs('bdd100k/train/images',exist_ok=True)
os.makedirs('bdd100k/train/labels',exist_ok=True)
os.makedirs('bdd100k/val/images',exist_ok=True)
os.makedirs('bdd100k/val/labels',exist_ok=True)
os.makedirs('bdd100k/test/images',exist_ok=True)"""


In [ ]:
#copy_files("/kaggle/input/hoangprj2/output/val/images","/kaggle/working/bdd100k/val/images")

In [ ]:
#copy_files("/kaggle/input/hoangprj2/output/val/labels","/kaggle/working/bdd100k/val/labels")
#copy_files("/kaggle/input/hoangprj2/output/test/images","/kaggle/working/bdd100k/test/images")
#copy_files("/kaggle/input/hoangprj2/output/train/images","/kaggle/working/bdd100k/train/images")
#copy_files("/kaggle/input/hoangprj2/output/train/labels","/kaggle/working/bdd100k/train/labels")

In [2]:
import yaml

In [3]:
class_labels=['person','car','rider','bus','truck','bike','motor','traffic light','traffic sign']
output_dir='bdd100k'
os.makedirs(output_dir, exist_ok=True)
data_yaml_path=os.path.join(output_dir,'bdd100_data.yml')
data_yaml={
    'path':'/kaggle/working/bdd100k',
    'train':'/kaggle/working/bdd100k/train/images',
    'val':'/kaggle/working/bdd100k/val/images',
    'nc':len(class_labels),
    'names':class_labels
    
    
}
with open(data_yaml_path,'w') as f:
    yaml.dump(data_yaml,f,default_flow_style=False)

In [4]:
!pip install ultralytics -q
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.28 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 5689.3/8062.4 GB disk)


In [5]:
from ultralytics import YOLO

In [6]:
import torch

In [7]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
!export CUDA_LAUNCH_BLOCKING=1


In [ ]:
'''model=YOLO('yolov8n.pt')
epochs=5
batch_size=-1 #Auto scaled base on GPU memory
img_size=(720,1080)
project_name='models/yolo'
name='yolov8s_bdd100k_det'

#Train the model
results=model.train(data=data_yaml_path,
                    epochs=epochs,
                    batch=batch_size,
                    imgsz=img_size,
                    project=project_name,
                    name=name)'''

# Build a function for using Yolo DeepSORT

In [9]:
class YOLOv8:
    def __init__(self,model_path):
        self.model=YOLO(model_path)
    def detect(self,source_img):
        results=self.model.predict(source_img,verbose=False)[0]
        bboxes=results.boxes.xywh.cpu().numpy()
        bboxes[:,:2]=bboxes[:,:2]-(bboxes[:,2:]/2)
        scores=results.boxes.conf.cpu().numpy()
        class_ids=results.boxes.cls.cpu().numpy()

        return bboxes,scores,class_ids

In [10]:
!git clone https://github.com/wjnwjn59/deep_sort.git

Cloning into 'deep_sort'...
remote: Enumerating objects: 167, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 167 (delta 9), reused 15 (delta 7), pack-reused 142
Receiving objects: 100% (167/167), 77.68 KiB | 1.08 MiB/s, done.
Resolving deltas: 100% (92/92), done.


In [11]:
!pip install gdown==4.6.0 -q

In [12]:
!gdown --no-check-certificate --folder https://drive.google.com/open?id=18fKzfqnqhqW3s9zwsCbnVJ5XF2JFeqMp

Retrieving folder list
Retrieving folder 1VVqtL0klSUvLnmBKS89il1EKC3IxUBVK detections
Retrieving folder 1qNWOpUtKG8GqEiL-LbBdXyvifUtcbOvc MOT16_POI_test
Processing file 1aEzvFHPK-N6hqLXMqhh3i9JJzn7WFUA3 MOT16-01.npy
Processing file 1h_ktJDBIEXaSBAA-RxKNYnL9e4fp2HPd MOT16-03.npy
Processing file 1ilOElwfYZLwQKH57HoYdXfuYhpPibfqF MOT16-06.npy
Processing file 1TajzH3GbumKmtYvKBvOtGERFGD0tStwG MOT16-07.npy
Processing file 1WB9Mi4RLVPHV4_20sVq7FdoeG5JYQ_J1 MOT16-08.npy
Processing file 1mksH9GWNT7zmcuq6rlRev8pevZz8Rfsm MOT16-12.npy
Processing file 1FVVhn_IpxQ_jkYhc0CUQHSQMm1SMTEBj MOT16-14.npy
Retrieving folder 1DcOcApOkxP3NdeIUXxVF1KNex6T6YDq3 MOT16_POI_train
Processing file 1Va__9NWU2ZCmaxIq4oIabi05NYWEOk1K MOT16-02.npy
Processing file 1EH7orgDPp7kqRY5OA0hEctcEtQnYq0Ea MOT16-04.npy
Processing file 1RCfHJx5ZoUecapbZCsgp0tCEiItvLsd8 MOT16-05.npy
Processing file 1VLOvn-mbpY0Q1rsMONQZhaEQIGEmyLQL MOT16-09.npy
Processing file 1SbMhOgYPvZ84xE8lRtXc7CLXJF86lwf4 MOT16-10.npy
Processing file 1a4w-Ho

In [13]:
from deep_sort.deep_sort import nn_matching
from deep_sort.deep_sort.detection import Detection
from deep_sort.deep_sort.tracker import Tracker 
from deep_sort.tools import generate_detections as gdet

### Define class DeepSORT

In [14]:
class DeepSORT:
    def __init__(self,model_path="/kaggle/working/resources/networks/mars-small128.pb",
                max_cosine_distance=0.7,
                nn_budget=None,classes=class_labels):
        self.encoder=gdet.create_box_encoder(model_path,batch_size=1)
        self.metric=nn_matching.NearestNeighborDistanceMetric('cosine',max_cosine_distance,nn_budget)
        self.tracker=Tracker(self.metric)
        key_list=[]
        val_list=[]
        for ID,class_name in enumerate(classes):
            key_list.append(ID)
            val_list.append(class_name)
        self.key_list=key_list
        self.val_list=val_list
    def tracking(self,origin_frame,bboxes,scores,class_ids):
        features=self.encoder(origin_frame,bboxes)
        detections=[Detection(bbox,score,class_id,feature)
                   for bbox,score,class_id,feature in zip(bboxes,scores,class_ids,features)]
        self.tracker.predict()
        self.tracker.update(detections)
        tracked_bboxes=[]
        for track in self.tracker.tracks:
            if not track.is_confirmed() or track.time_since_update>5:
                continue
            bbox=track.to_tlbr()
            class_id=track.get_class()
            conf_score=track.get_conf_score()
            tracking_id=track.track_id
            tracked_bboxes.append(
                bbox.tolist()+[class_id,conf_score,tracking_id])
        tracked_bboxes=np.array(tracked_bboxes)
        return tracked_bboxes

In [15]:
def draw_detection(img,bboxes,scores,class_ids,ids,classes=class_labels,mask_alpha=0.3):
    height,width=img.shape[:2]
    np.random.seed(0)
    rng=np.random.default_rng(3)
    colors=rng.uniform(0,255,size=(len(classes),3))
    mask_img=img.copy()
    det_img=img.copy()
    size=min([height,width])*0.0006
    text_thickness=int(min([height,width])*0.001)
    for bbox,score,class_id,id_ in zip(bboxes,scores,class_ids,ids):
        color=colors[class_id]
        x1,y1,x2,y2=bbox.astype(int)
        #Draw rectangle
        cv2.rectangle(det_img,(x1,y1),(x2,y2),color,2)
        cv2.rectangle(mask_img,(x1,y1),(x2,y2),color,-1)
        label=classes[class_id]
        caption=f'{label} {int(score*100)}% ID: {id_}'
        (tw,th),_=cv2.getTextSize(text=caption,
                                 fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                                 fontScale=size,
                                 thickness=text_thickness)
        
        th=int(th*1.2)
        cv2.rectangle(det_img,(x1,y1),(x1+tw,y1-th),color,-1)
        cv2.rectangle(mask_img,(x1,y1),(x1+tw,y1-th),color,-1)
        cv2.putText(det_img,caption,(x1,y1),
                   cv2.FONT_HERSHEY_SIMPLEX,size,
                   (255,255,255),
                   text_thickness,cv2.LINE_AA)
        cv2.putText(mask_img,caption,(x1,y1),
                   cv2.FONT_HERSHEY_SIMPLEX,size,
                   (255,255,255),
                   text_thickness,cv2.LINE_AA)
    return cv2.addWeighted(mask_img,mask_alpha,det_img,1-mask_alpha,0)

In [16]:
import time
def video_tracking(video_path, detector, tracker, is_save_result=False, save_dir='tracking_result'):
    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    if is_save_result:
        os.makedirs(save_dir, exist_ok=True)
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        save_result_name = 'output_video.mp4'
        save_result_path = os.path.join(save_dir, save_result_name)
        out = cv2.VideoWriter(save_result_path, fourcc, fps, (width, height))
    
    tracking_results = {}
    tracked_ids = np.array([], dtype=np.int32)
    frame_number = 0
    total_time = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        start_time = time.time()
        
        detector_results = detector.detect(frame)
        bboxes, scores, class_ids = detector_results
        tracker_pred = tracker.tracking(origin_frame=frame, bboxes=bboxes, scores=scores, class_ids=class_ids)
        
        frame_time = time.time() - start_time
        total_time += frame_time
        tracking_results[frame_number]=[]
        if tracker_pred.size > 0:
            bboxes = tracker_pred[:, :4]
            class_ids = tracker_pred[:, 4].astype(np.int32)
            conf_scores = tracker_pred[:, 5]
            tracking_ids = tracker_pred[:, 6].astype(np.int32)
            new_ids = np.setdiff1d(tracking_ids, tracked_ids)
            tracked_ids = np.concatenate((tracked_ids, new_ids))
            result_img = draw_detection(img=frame,
                                        bboxes=bboxes,
                                        scores=conf_scores,
                                        class_ids=class_ids,
                                        ids=tracking_ids)
            for bbox, track_id, class_id, score in zip(bboxes, tracking_ids, class_ids, conf_scores):
                tracking_results[frame_number].append({
                    "track_id": track_id,
                    "class_id": class_id,
                    "x_min": bbox[0],
                    "y_min": bbox[1],
                    "x_max": bbox[2],
                    "y_max": bbox[3],
                    "confidence": score
                })
        else:
            result_img = frame
        
        
        frame_number += 1
        if is_save_result:
            out.write(result_img)
        
        # Print frame processing time
        print(f"Frame {frame_number} processed in {frame_time:.4f} seconds")
    
    cap.release()
    if is_save_result:
        out.release()
    
    # Print total processing time
    print(f"Total processing time: {total_time:.4f} seconds")
    
    return tracking_results

In [17]:
yolo_model_path="/kaggle/input/checkcheckcheck/best (1).pt"
detector=YOLOv8(yolo_model_path)
tracker=DeepSORT()
video_path="/kaggle/input/driving-video-with-object-tracking/bdd100k_videos_train_00/bdd100k/videos/train/0000f77c-6257be58.mov"
deepsort_results=video_tracking(video_path,detector,tracker,is_save_result=True)

Frame 1 processed in 5.7592 seconds
Frame 2 processed in 0.1005 seconds
Frame 3 processed in 0.0305 seconds
Frame 4 processed in 0.0350 seconds
Frame 5 processed in 0.0308 seconds
Frame 6 processed in 0.0315 seconds
Frame 7 processed in 0.0282 seconds
Frame 8 processed in 0.0298 seconds
Frame 9 processed in 0.0314 seconds
Frame 10 processed in 0.0301 seconds
Frame 11 processed in 0.0325 seconds
Frame 12 processed in 0.0336 seconds
Frame 13 processed in 0.0317 seconds
Frame 14 processed in 0.0293 seconds
Frame 15 processed in 0.0320 seconds
Frame 16 processed in 0.0300 seconds
Frame 17 processed in 0.0328 seconds
Frame 18 processed in 0.0301 seconds
Frame 19 processed in 0.0357 seconds
Frame 20 processed in 0.0314 seconds
Frame 21 processed in 0.0293 seconds
Frame 22 processed in 0.0281 seconds
Frame 23 processed in 0.0328 seconds
Frame 24 processed in 0.0271 seconds
Frame 25 processed in 0.0276 seconds
Frame 26 processed in 0.0280 seconds
Frame 27 processed in 0.0297 seconds
Frame 28 p

In [23]:
deepsort_id=set()
for i in range(len(deepsort_results)):
    if deepsort_results[i] is not None:
        for object in deepsort_results[i]:
            deepsort_id.add(object['track_id'])
print(len(deepsort_id))

289


In [78]:
category_mapping = {
    'car': 'car',
    'pedestrian': 'person',
    'truck': 'truck',
    'bus': 'bus',
    'bicycle': 'bike',
    'rider': 'rider',
    'motorcycle': 'motor',
    'traffic light': 'traffic light',
    'traffic sign': 'traffic sign',
    'other person': 'person'
}


In [79]:
labels_df=pd.read_csv("/kaggle/input/driving-video-with-object-tracking/mot_labels.csv",low_memory=False)

In [80]:
root="/kaggle/input/driving-video-with-object-tracking/bdd100k_videos_train_00/bdd100k/videos/train"
videoName="0000f77c-6257be58"
def filter_and_map_ground_truths(labels_df, category_mapping, video_name):
    filtered_ground_truths = []
    video_path = os.path.join(root, f"{video_name}.mov")  
    cap = cv2.VideoCapture(video_path)
    image_width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    image_height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    fps = round(cap.get(cv2.CAP_PROP_FPS))
    if image_width == 0 or image_height == 0 or fps == 0:
        raise ValueError("Failed to get video properties. Ensure the video file is correct.")
    
    cap.release()
    extracted_df=labels_df[labels_df["videoName"]==videoName]
    for index, row in extracted_df.iterrows():
        
        if row['category'] in category_mapping:
            mapped_category = category_mapping[row['category']]   
            # Convert (x1, y1, x2, y2) to (x_center, y_center, width, height)
            x1 = row['box2d.x1']
            y1 = row['box2d.y1']
            x2 = row['box2d.x2']
            y2 = row['box2d.y2']
            row['category'] = mapped_category
            row['trueFrame'] = int(row['frameIndex'] *fps / 5-1)
            filtered_ground_truths.append(row)
    print(fps)
    
    return pd.DataFrame(filtered_ground_truths)

extracted_df=filter_and_map_ground_truths(labels_df, category_mapping, videoName)

30


In [98]:
extracted_df[extracted_df['trueFrame']==5]

,name,videoName,frameIndex,id,category,attributes.crowd,attributes.occluded,attributes.truncated,box2d.x1,box2d.x2,box2d.y1,box2d.y2,haveVideo,trueFrame
1506825,0000f77c-6257be58-0000002.jpg,0000f77c-6257be58,1,37555.0,car,False,False,False,609.318047,977.532372,211.411012,532.720720,True,5
1506826,0000f77c-6257be58-0000002.jpg,0000f77c-6257be58,1,37556.0,car,False,False,True,1130.360726,1278.674291,247.964932,518.760884,True,5


In [82]:
output_dir="/kaggle/working/extract"
os.makedirs(output_dir, exist_ok=True)

In [83]:
videoName="0000f77c-6257be58"

In [87]:
def extract_and_label_frames(video_path, labels_df, output_dir):
    cap = cv2.VideoCapture(video_path)
    output_dir_images = os.path.join(output_dir, 'images')
    output_dir_labels = os.path.join(output_dir, 'labels')
    os.makedirs(output_dir_images, exist_ok=True)
    os.makedirs(output_dir_labels, exist_ok=True)

    frameIndex = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_labels = labels_df[labels_df['trueFrame'] == frameIndex]
        if not frame_labels.empty:
            # Save the image
            filename = f"{os.path.basename(video_path).split('.')[0]}_{frameIndex:06d}.jpg"
            file_path = os.path.join(output_dir_images, filename)
            cv2.imwrite(file_path, frame)

            # Save the labels
            txt_file_path = os.path.join(output_dir_labels, filename.replace('.jpg', '.txt'))
            with open(txt_file_path, 'w') as f:
                for _, label_row in frame_labels.iterrows():
                    object_class = label_row['category']
                    class_id = class_labels.index(object_class)
                    x_min = label_row['box2d.x1']
                    y_min = label_row['box2d.y1']
                    x_max = label_row['box2d.x2']
                    y_max = label_row['box2d.y2']
                    track_id = int(label_row['id'])
                    f.write(f"{track_id} {class_id} {x_min} {y_min} {x_max} {y_max}\n")
        
        frameIndex += 1

    cap.release()

video_path = os.path.join(root, f"{videoName}.mov") 
extract_and_label_frames(video_path, extracted_df, output_dir)

In [88]:
print(len(os.listdir("/kaggle/working/extract/images")))

202


In [91]:
print((os.listdir("/kaggle/working/extract/labels")))

['0000f77c-6257be58_000155.txt', '0000f77c-6257be58_001139.txt', '0000f77c-6257be58_000449.txt', '0000f77c-6257be58_000371.txt', '0000f77c-6257be58_001205.txt', '0000f77c-6257be58_000185.txt', '0000f77c-6257be58_000833.txt', '0000f77c-6257be58_000935.txt', '0000f77c-6257be58_000731.txt', '0000f77c-6257be58_001061.txt', '0000f77c-6257be58_000953.txt', '0000f77c-6257be58_000377.txt', '0000f77c-6257be58_000227.txt', '0000f77c-6257be58_000815.txt', '0000f77c-6257be58_000353.txt', '0000f77c-6257be58_000557.txt', '0000f77c-6257be58_001007.txt', '0000f77c-6257be58_001049.txt', '0000f77c-6257be58_001169.txt', '0000f77c-6257be58_000179.txt', '0000f77c-6257be58_000029.txt', '0000f77c-6257be58_000215.txt', '0000f77c-6257be58_000761.txt', '0000f77c-6257be58_000599.txt', '0000f77c-6257be58_001067.txt', '0000f77c-6257be58_000611.txt', '0000f77c-6257be58_000683.txt', '0000f77c-6257be58_000257.txt', '0000f77c-6257be58_001127.txt', '0000f77c-6257be58_000209.txt', '0000f77c-6257be58_000221.txt', '0000f7

In [97]:
file_path="/kaggle/working/extract/labels/0000f77c-6257be58_000005.txt"
f=open(file_path)
lines=f.readlines()
for line in lines:
    print(line)

37555 1 609.3180467751063 211.4110122910357 977.532372194142 532.720720028821

37556 1 1130.360725953617 247.9649316933764 1278.6742905076617 518.7608837190613



In [93]:
frame_numbers=[]
for label_file in os.listdir("/kaggle/working/extract/labels"):
    frame_number = int(label_file.split('_')[-1].split('.')[0])
    frame_numbers.append(frame_number)
    
min_frame_number = min(frame_numbers)
print(min_frame_number)
print(len(frame_numbers))

5
202


In [94]:
def load_ground_truths(labels_dir):
    ground_truths = {}
    for label_file in os.listdir(labels_dir):
        frame_number = int(int(label_file.split('_')[-1].split('.')[0]))
        with open(os.path.join(labels_dir, label_file), 'r') as f:
            objects = []
            for line in f:
                parts = line.strip().split()
                track_id = int(parts[0])
                class_id = int(parts[1])
                x_min = float(parts[2])
                y_min = float(parts[3])
                x_max = float(parts[4])
                y_max = float(parts[5])
                objects.append({
                    'track_id': track_id,
                    'class_id': class_id,
                    'x_min': x_min,
                    'y_min': y_min,
                    'x_max': x_max,
                    'y_max': y_max
                })
            ground_truths[frame_number] = objects
    return ground_truths

In [95]:
ground_truths=load_ground_truths("/kaggle/working/extract/labels")

In [96]:
ground_truths[5]

[{'track_id': 37555,
  'class_id': 1,
  'x_min': 609.3180467751063,
  'y_min': 211.4110122910357,
  'x_max': 977.532372194142,
  'y_max': 532.720720028821},
 {'track_id': 37556,
  'class_id': 1,
  'x_min': 1130.360725953617,
  'y_min': 247.9649316933764,
  'x_max': 1278.6742905076617,
  'y_max': 518.7608837190613}]

# ByteTrack

In [ ]:
!git clone https://github.com/jahongir7174/ByteTrack.git

In [24]:
!pip install motmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.5/161.5 kB 1.6 MB/s eta 0:00:00 0:00:01


In [25]:
!pip install lap


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 7.0 MB/s eta 0:00:0000:0100:010m
  Preparing metadata (setup.py) ... done
  Created wheel for lap: filename=lap-0.4.0-cp310-cp310-linux_x86_64.whl size=1483137 sha256=dfee7f19c1daa0e3942cb1daaf751eb327a38eeaa50f0f0edf0402a2e295bc55
  Stored in directory: /root/.cache/pip/wheels/00/42/2e/9dfe19270eea279d79e84767ff0d7b8082c3bf776cad00e83d
Successfully built lap


In [26]:
import time

import cv2
import numpy
import scipy
import torch
import torchvision

try:
    import lap  # for linear_assignment

    assert lap.__version__  # verify package is not directory
except (ImportError, AssertionError, AttributeError):
    import lap

In [ ]:
!pip install supervision==0.13.0
     

In [ ]:
from ultralytics import YOLO
from tqdm.notebook import tqdm
from supervision.geometry.core import Point
from supervision.utils.video import VideoSink
from supervision.utils.video import VideoInfo
from supervision.utils.notebook import plot_image
from supervision.detection.core import Detections
from supervision.detection.annotate import BoxAnnotator
from supervision.draw.color import Color, ColorPalette
from supervision.tracker.byte_tracker.core import ByteTrack
from supervision.utils.video import get_video_frames_generator
from supervision.detection.line_counter import LineZone, LineZoneAnnotator

In [69]:
model = YOLO('/kaggle/input/checkcheckcheck/best (1).pt')  # weights from trained model

# Open the video file
video_path = r"/kaggle/input/driving-video-with-object-tracking/bdd100k_videos_train_00/bdd100k/videos/train/0000f77c-6257be58.mov"
cap = cv2.VideoCapture(video_path)
VIDEO_CODEC = "mp4v"
fps = round(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter("out_byte.mp4",
                      cv2.VideoWriter_fourcc(*VIDEO_CODEC),
                      fps,
                      (width, height))

total_time_preprocess = 0
total_time_inference = 0
total_time_postprocess = 0
total_frame_time=0
frame_number = 0
total1=[]
# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        start_time = time.time()

        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist=True, show=False, tracker="bytetrack.yaml")
        total1.append(results)
        # Extract timing information from results
        timing_info = results[0].speed  # speed dictionary with preprocess, inference, postprocess times
        preprocess_time = timing_info['preprocess']
        inference_time = timing_info['inference']
        postprocess_time = timing_info['postprocess']

        # Accumulate the times
        total_time_preprocess += preprocess_time
        total_time_inference += inference_time
        total_time_postprocess += postprocess_time

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Display the annotated frame
        out.write(annotated_frame)

        frame_number += 1
        frame_time = time.time() - start_time
        total_frame_time+=frame_time
        # Print frame processing time
        print(f"Frame {frame_number} processed in {frame_time:.4f} seconds (Preprocess: {preprocess_time:.4f}, Inference: {inference_time:.4f}, Postprocess: {postprocess_time:.4f})")

    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
out.release()
cap.release()


# Print total processing times
print(f"Total preprocess time: {total_time_preprocess:.4f} seconds")
print(f"Total inference time: {total_time_inference:.4f} seconds")
print(f"Total postprocess time: {total_time_postprocess:.4f} seconds")
print(f"Total processing time: {total_frame_time:.4f} seconds")


0: 640x1088 2 cars, 2 traffic signs, 11.5ms
Speed: 4.2ms preprocess, 11.5ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 1088)
Frame 1 processed in 0.1536 seconds (Preprocess: 4.1697, Inference: 11.4756, Postprocess: 1.5502)

0: 640x1088 2 cars, 2 traffic signs, 10.9ms
Speed: 5.2ms preprocess, 10.9ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 1088)
Frame 2 processed in 0.0330 seconds (Preprocess: 5.1620, Inference: 10.8883, Postprocess: 1.1988)

0: 640x1088 2 cars, 1 traffic ligt, 2 traffic signs, 10.9ms
Speed: 5.1ms preprocess, 10.9ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 1088)
Frame 3 processed in 0.0346 seconds (Preprocess: 5.0559, Inference: 10.8957, Postprocess: 1.1885)

0: 640x1088 2 cars, 2 traffic signs, 10.9ms
Speed: 5.1ms preprocess, 10.9ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 1088)
Frame 4 processed in 0.0324 seconds (Preprocess: 5.1453, Inference: 10.9057, Postprocess: 1.1873)

0: 640x1088 2 cars,

In [72]:
bytetrack_results={}
for i in range(len(total1)):
    bytetrack_results[i]=[]
    for obj in total1[i][0].boxes:
        box=obj.xyxy[0]
        class_id=int(obj.cls[0])
        confidence=float(obj.conf[0])
        track_id=int(obj.id)
        bytetrack_results[i].append({
            'track_id':track_id,
            'class_id':class_id,
            'x_min':box[0],
            'y_min':box[1],
            'x_max':box[2],
            'y_max':box[3],
            'confidence':confidence
        })
        
        

In [73]:
bytetrack_id=set()
for i in range(len(deepsort_results)):
    if bytetrack_results[i] is not None:
        for object in bytetrack_results[i]:
            bytetrack_id.add(object['track_id'])
print(len(bytetrack_id))

111


In [74]:
bytetrack_results[1]

[{'track_id': 1,
  'class_id': 1,
  'x_min': tensor(609.0618),
  'y_min': tensor(217.7179),
  'x_max': tensor(972.5645),
  'y_max': tensor(527.5377),
  'confidence': 0.9540819525718689},
 {'track_id': 2,
  'class_id': 1,
  'x_min': tensor(1129.6030),
  'y_min': tensor(230.9695),
  'x_max': tensor(1280.),
  'y_max': tensor(586.9293),
  'confidence': 0.814551830291748},
 {'track_id': 3,
  'class_id': 8,
  'x_min': tensor(1062.1283),
  'y_min': tensor(249.2529),
  'x_max': tensor(1113.3729),
  'y_max': tensor(294.6359),
  'confidence': 0.7754557132720947},
 {'track_id': 4,
  'class_id': 8,
  'x_min': tensor(1136.9835),
  'y_min': tensor(241.3206),
  'x_max': tensor(1174.1990),
  'y_max': tensor(275.0953),
  'confidence': 0.703913152217865}]

# BotSORT

In [ ]:
from collections import defaultdict

In [35]:
model = YOLO('/kaggle/input/checkcheckcheck/best (1).pt')  # weights from trained model

# Open the video file
video_path = r"/kaggle/input/driving-video-with-object-tracking/bdd100k_videos_train_00/bdd100k/videos/train/0000f77c-6257be58.mov"
cap = cv2.VideoCapture(video_path)
VIDEO_CODEC = "mp4v"
fps = round(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter("out_test.mp4",
                      cv2.VideoWriter_fourcc(*VIDEO_CODEC),
                      fps,
                      (width, height))

total_time_preprocess = 0
total_time_inference = 0
total_time_postprocess = 0
total_frame_time=0
frame_number = 0
total1=[]
# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        start_time = time.time()

        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist=True, show=False, tracker="botsort.yaml")
        total1.append(results)
        # Extract timing information from results
        timing_info = results[0].speed  # speed dictionary with preprocess, inference, postprocess times
        preprocess_time = timing_info['preprocess']
        inference_time = timing_info['inference']
        postprocess_time = timing_info['postprocess']

        # Accumulate the times
        total_time_preprocess += preprocess_time
        total_time_inference += inference_time
        total_time_postprocess += postprocess_time

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Display the annotated frame
        out.write(annotated_frame)

        frame_number += 1
        frame_time = time.time() - start_time
        total_frame_time+=frame_time
        # Print frame processing time
        print(f"Frame {frame_number} processed in {frame_time:.4f} seconds (Preprocess: {preprocess_time:.4f}, Inference: {inference_time:.4f}, Postprocess: {postprocess_time:.4f})")

    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
out.release()
cap.release()


# Print total processing times
print(f"Total preprocess time: {total_time_preprocess:.4f} seconds")
print(f"Total inference time: {total_time_inference:.4f} seconds")
print(f"Total postprocess time: {total_time_postprocess:.4f} seconds")
print(f"Total processing time: {total_frame_time:.4f} seconds")


0: 640x1088 2 cars, 2 traffic signs, 12.5ms
Speed: 5.0ms preprocess, 12.5ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 1088)
Frame 1 processed in 0.2342 seconds (Preprocess: 4.9667, Inference: 12.4621, Postprocess: 2.0816)

0: 640x1088 2 cars, 2 traffic signs, 11.0ms
Speed: 6.4ms preprocess, 11.0ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 1088)
Frame 2 processed in 0.0816 seconds (Preprocess: 6.3930, Inference: 10.9539, Postprocess: 1.3082)

0: 640x1088 2 cars, 1 traffic ligt, 2 traffic signs, 10.9ms
Speed: 5.8ms preprocess, 10.9ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 1088)
Frame 3 processed in 0.0652 seconds (Preprocess: 5.8382, Inference: 10.9394, Postprocess: 1.2774)

0: 640x1088 2 cars, 1 traffic ligt, 2 traffic signs, 11.0ms
Speed: 6.4ms preprocess, 11.0ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 1088)
Frame 4 processed in 0.0666 seconds (Preprocess: 6.4199, Inference: 10.9925, Postprocess: 1.3518)

0: 

In [60]:
botsort_results={}
for i in range(len(total1)):
    botsort_results[i]=[]
    for obj in total1[i][0].boxes:
        box=obj.xyxy[0]
        class_id=int(obj.cls[0])
        confidence=float(obj.conf[0])
        track_id=int(obj.id)
        botsort_results[i].append({
            'track_id':track_id,
            'class_id':class_id,
            'x_min':box[0],
            'y_min':box[1],
            'x_max':box[2],
            'y_max':box[3],
            'confidence':confidence
        })
        
        

In [63]:
botsort_results[1216]

[{'track_id': 77,
  'class_id': 1,
  'x_min': tensor(696.6570),
  'y_min': tensor(304.3178),
  'x_max': tensor(856.3178),
  'y_max': tensor(428.3143),
  'confidence': 0.9245055317878723},
 {'track_id': 145,
  'class_id': 8,
  'x_min': tensor(910.1976),
  'y_min': tensor(220.1995),
  'x_max': tensor(953.6256),
  'y_max': tensor(272.4688),
  'confidence': 0.8454720377922058},
 {'track_id': 146,
  'class_id': 8,
  'x_min': tensor(968.1200),
  'y_min': tensor(209.8988),
  'x_max': tensor(1026.0312),
  'y_max': tensor(264.7005),
  'confidence': 0.8571966886520386},
 {'track_id': 148,
  'class_id': 1,
  'x_min': tensor(1003.5430),
  'y_min': tensor(188.1347),
  'x_max': tensor(1279.3689),
  'y_max': tensor(548.7388),
  'confidence': 0.9392483234405518},
 {'track_id': 147,
  'class_id': 8,
  'x_min': tensor(1037.1709),
  'y_min': tensor(201.6383),
  'x_max': tensor(1098.3480),
  'y_max': tensor(259.9595),
  'confidence': 0.7701995372772217},
 {'track_id': 150,
  'class_id': 1,
  'x_min': tens

In [67]:
botsort_id=set()
for i in range(len(botsort_results)):
    if botsort_results[i] is not None:
        for object in botsort_results[i]:
            botsort_id.add(object['track_id'])
print(len(botsort_id))

110


In [68]:
len(botsort_results)

1217

In [75]:
print(len(deepsort_results),len(botsort_results),len(bytetrack_results))

1217 1217 1217


# Evaluate

In [76]:
from collections import defaultdict

def compute_iou(box1, box2):
    x1_min, y1_min, x1_max, y1_max = box1
    x2_min, y2_min, x2_max, y2_max = box2

    intersect_min_x = max(x1_min, x2_min)
    intersect_min_y = max(y1_min, y2_min)
    intersect_max_x = min(x1_max, x2_max)
    intersect_max_y = min(y1_max, y2_max)

    intersect_area = max(0, intersect_max_x - intersect_min_x) * max(0, intersect_max_y - intersect_min_y)

    area1 = (x1_max - x1_min) * (y1_max - y1_min)
    area2 = (x2_max - x2_min) * (y2_max - y2_min)

    union_area = area1 + area2 - intersect_area

    if union_area == 0:
        return 0

    return intersect_area / union_area

In [77]:
def count_ids_switches(tracking_results, iou_threshold=0.7):
    id_switches = 0
    previous_frame_objects = {}

    for frame_id in sorted(tracking_results.keys()):
        current_frame_objects = tracking_results[frame_id]
        current_frame_objects_dict = {obj['track_id']: obj for obj in current_frame_objects}

        # Compare with previous frame objects
        for track_id, current_obj in current_frame_objects_dict.items():
            best_iou = 0
            best_prev_track_id = None
            for prev_track_id, prev_obj in previous_frame_objects.items():
                iou = compute_iou(
                    [current_obj['x_min'], current_obj['y_min'], current_obj['x_max'], current_obj['y_max']],
                    [prev_obj['x_min'], prev_obj['y_min'], prev_obj['x_max'], prev_obj['y_max']]
                )
                if iou > best_iou:
                    best_iou = iou
                    best_prev_track_id = prev_track_id

            if best_iou > iou_threshold and best_prev_track_id is not None:
                if track_id != best_prev_track_id:
                    id_switches += 1

        previous_frame_objects = current_frame_objects_dict

    return id_switches

id_switches_deepsort = count_ids_switches(deepsort_results)
id_switches_bytrack = count_ids_switches(bytetrack_results)
id_switches_botsort = count_ids_switches(botsort_results)

print(f"ID switches for DeepSORT: {id_switches_deepsort}")
print(f"ID switches for ByteTrack: {id_switches_bytrack}")
print(f"ID switches for BotSORT: {id_switches_botsort}")


ID switches for DeepSORT: 124
ID switches for ByteTrack: 2
ID switches for BotSORT: 3


In [100]:
def extract_frames_at_5hz(video_path, fps):
    cap = cv2.VideoCapture(video_path)
    frames = []
    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if (frame_count+1) % (fps // 5) == 0:
            frames.append(frame_count)
        frame_count += 1
    cap.release()
    return frames
def filter_tracking_results_at_5hz(tracking_results, frames_5hz):
    filtered_tracking_results = {}
    for frame_number in frames_5hz:
        if frame_number in tracking_results:
            filtered_tracking_results[frame_number] = tracking_results[frame_number]
    return filtered_tracking_results

In [104]:
frames_5hz=extract_frames_at_5hz(video_path, fps)

In [108]:
filter_deepsort=filter_tracking_results_at_5hz(deepsort_results, frames_5hz)
filter_bytetrack=filter_tracking_results_at_5hz(bytetrack_results,frames_5hz)
filter_botsort=filter_tracking_results_at_5hz(botsort_results,frames_5hz)

In [115]:
import motmetrics as mm
import numpy as np
from tqdm import tqdm

def evaluate_with_motmetrics(tracking_results, ground_truths):
    acc = mm.MOTAccumulator(auto_id=True)
    
    for frame_number in tqdm(tracking_results.keys()):
        detections = tracking_results[frame_number]
        if frame_number in ground_truths:
            gt_bboxes = ground_truths[frame_number]
            if len(detections) > 0:
                bboxes = np.array([[det['x_min'], det['y_min'], det['x_max'], det['y_max']] for det in detections])
                gt_boxes = np.array([[gt['x_min'], gt['y_min'], gt['x_max'], gt['y_max']] for gt in gt_bboxes])
                class_ids = np.array([det['class_id'] for det in detections])
                track_ids = np.array([det['track_id'] for det in detections])
                gt_ids = np.array([gt['track_id'] for gt in gt_bboxes])
                distance_matrix = mm.distances.iou_matrix(gt_boxes, bboxes, max_iou=0.5)
                acc.update(gt_ids, track_ids, distance_matrix)
            else:
                gt_ids = np.array([gt['track_id'] for gt in gt_bboxes])
                acc.update(gt_ids, [], np.zeros((len(gt_bboxes), 0)))
        elif len(detections) > 0:
            track_ids = np.array([det['track_id'] for det in detections])
            acc.update([], track_ids, np.zeros((0, len(detections))))
    
    mh = mm.metrics.create()
    summary = mh.compute(acc, metrics=['num_frames', 'num_objects', 'num_matches', 'mota', 'motp', 'idf1', 'num_misses', 'num_false_positives', 'num_switches', 'mostly_tracked', 'partially_tracked', 'mostly_lost'], name='acc')
    print(summary)

In [114]:
gt_bboxes = np.array(ground_truths[5])
gt_bboxes

array([{'track_id': 37555, 'class_id': 1, 'x_min': 609.3180467751063, 'y_min': 211.4110122910357, 'x_max': 977.532372194142, 'y_max': 532.720720028821}, {'track_id': 37556, 'class_id': 1, 'x_min': 1130.360725953617, 'y_min': 247.9649316933764, 'x_max': 1278.6742905076617, 'y_max': 518.7608837190613}], dtype=object)

In [112]:
detections = np.array(filter_deepsort[5])
detections

array([{'track_id': 1, 'class_id': 1, 'x_min': 593.3418410370508, 'y_min': 207.26215697368988, 'x_max': 992.2001030380404, 'y_max': 547.4083496651486, 'confidence': 0.9550378322601318},
       {'track_id': 2, 'class_id': 1, 'x_min': 1115.9048339892029, 'y_min': 211.53013511394354, 'x_max': 1280.6812575300949, 'y_max': 600.7247744877352, 'confidence': 0.820733904838562},
       {'track_id': 3, 'class_id': 8, 'x_min': 1065.0412247328484, 'y_min': 252.81093054230226, 'x_max': 1117.7064158413068, 'y_max': 299.51645197646764, 'confidence': 0.7752721905708313},
       {'track_id': 4, 'class_id': 8, 'x_min': 1148.8952521028639, 'y_min': 244.8305872560084, 'x_max': 1187.6331113102708, 'y_max': 279.8018706689936, 'confidence': 0.7100333571434021},
       {'track_id': 5, 'class_id': 7, 'x_min': 1196.9816959922578, 'y_min': 263.32722001974713, 'x_max': 1206.6713786112045, 'y_max': 281.23140201710237, 'confidence': 0.5125088691711426},
       {'track_id': 6, 'class_id': 7, 'x_min': 947.07226341070

In [116]:
# Evaluate the tracking results with MOTMetrics
print("Evaluating DeepSORT:")
evaluate_with_motmetrics(filter_deepsort, ground_truths)

Evaluating DeepSORT:


100%|██████████| 202/202 [00:00<00:00, 1325.89it/s]


     num_frames  num_objects  num_matches      mota     motp      idf1  \
acc         202         1124          941 -0.524911  0.06873  0.424851   

     num_misses  num_false_positives  num_switches  mostly_tracked  \
acc          93                 1531            90              39   

     partially_tracked  mostly_lost  
acc                  5            3  


In [117]:
print("Evaluating ByteTrack:")
evaluate_with_motmetrics(filter_bytetrack, ground_truths)


Evaluating ByteTrack:


100%|██████████| 202/202 [00:00<00:00, 1116.26it/s]


     num_frames  num_objects  num_matches      mota      motp      idf1  \
acc         202         1124          895  0.174377  0.074113  0.551599   

     num_misses  num_false_positives  num_switches  mostly_tracked  \
acc         195                  699            34              31   

     partially_tracked  mostly_lost  
acc                  8            8  


In [118]:

print("Evaluating BotSORT:")
evaluate_with_motmetrics(filter_botsort, ground_truths) 

Evaluating BotSORT:


100%|██████████| 202/202 [00:00<00:00, 1113.55it/s]


     num_frames  num_objects  num_matches      mota      motp      idf1  \
acc         202         1124          920  0.181495  0.070367  0.561404   

     num_misses  num_false_positives  num_switches  mostly_tracked  \
acc         171                  716            33              32   

     partially_tracked  mostly_lost  
acc                  9            6  
